In [4]:
import pygame
import random

pygame 2.0.0 (SDL 2.0.12, python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
pygame.init()

(6, 0)

In [6]:
# window height and width
winHeight = 480
winWidth = 700
win=pygame.display.set_mode((winWidth,winHeight))

In [7]:
# colors RGB values  
BLACK = (0,0, 0)
WHITE = (255,255,255)
LIGHT_BLUE = (102,255,255)

In [8]:
btn_font = pygame.font.SysFont("arial", 20)
guess_font = pygame.font.SysFont("monospace", 24)
lost_font = pygame.font.SysFont('arial', 45)
word = ''
buttons = []
guessed = []

limbs = 0

In [9]:
picturesdir = "Pictures/"
import os
filenames = os.listdir(picturesdir)
hangmanPics = []
# load all images
for img in filenames:
    hangmanPics.append(pygame.image.load(picturesdir+img))

In [10]:
def redraw_game_window():
    global guessed
    global hangmanPics
    global limbs
    win.fill(LIGHT_BLUE)
    # Buttons
    for i in range(len(buttons)):
        if buttons[i][4]:
            pygame.draw.circle(win, BLACK, (buttons[i][1], buttons[i][2]), buttons[i][3])
            pygame.draw.circle(win, buttons[i][0], (buttons[i][1], buttons[i][2]), buttons[i][3] - 2
                               )
            label = btn_font.render(chr(buttons[i][5]), 1, BLACK)
            win.blit(label, (buttons[i][1] - (label.get_width() / 2), buttons[i][2] - (label.get_height() / 2)))

    spaced = spacedOut(word, guessed)
    label1 = guess_font.render(spaced, 1, BLACK)
    rect = label1.get_rect()
    length = rect[2]
    
    win.blit(label1,(winWidth/2 - length/2, 400))

    pic = hangmanPics[limbs]
    win.blit(pic, (winWidth/2 - pic.get_width()/2 + 20, 150))
    pygame.display.update()

In [11]:
def randomWord():
    file = open('words.txt')
    f = file.readlines()
    i = random.randrange(0, len(f) - 1)

    return f[i][:-1]

In [12]:
def hang(guess):
    global word
    if guess.lower() not in word.lower():
        return True
    else:
        return False

In [13]:
def spacedOut(word, guessed=[]):
    spacedWord = ''
    guessedLetters = guessed
    for x in range(len(word)):
        if word[x] != ' ':
            spacedWord += '_ '
            for i in range(len(guessedLetters)):
                if word[x].upper() == guessedLetters[i]:
                    spacedWord = spacedWord[:-2]
                    spacedWord += word[x].upper() + ' '
        elif word[x] == ' ':
            spacedWord += ' '
    return spacedWord

In [14]:
def buttonHit(x, y):
    for i in range(len(buttons)):
        if x < buttons[i][1] + 20 and x > buttons[i][1] - 20:
            if y < buttons[i][2] + 20 and y > buttons[i][2] - 20:
                return buttons[i][5]
    return None

In [15]:
def end(winner=False):
    global limbs
    lostTxt = 'You Lost, press any key to play again...'
    winTxt = 'You Win! press any key to play again...'
    redraw_game_window()
    pygame.time.delay(1000)
    win.fill(LIGHT_BLUE)

    if winner == True:
        label = lost_font.render(winTxt, 1, BLACK)
    else:
        label = lost_font.render(lostTxt, 1, BLACK)

    wordTxt = lost_font.render(word.upper(), 1, BLACK)
    wordWas = lost_font.render('The answer was: ', 1, BLACK)

    win.blit(wordTxt, (winWidth/2 - wordTxt.get_width()/2, 295))
    win.blit(wordWas, (winWidth/2 - wordWas.get_width()/2, 245))
    win.blit(label, (winWidth / 2 - label.get_width() / 2, 140))
    pygame.display.update()
    again = True
    while again:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            if event.type == pygame.KEYDOWN:
                again = False
    reset()

In [16]:
def reset():
    global limbs
    global guessed
    global buttons
    global word
    for i in range(len(buttons)):
        buttons[i][4] = True

    limbs = 0
    guessed = []
    word = randomWord()

In [17]:
# Setup buttons
increase = round(winWidth / 13)

for i in range(26):
    if i < 13:
        y = 40
        x = 25 + (increase * i)
    else:
        x = 25 + (increase * (i - 13))
        y = 85
    buttons.append([WHITE, x, y, 20, True, 65 + i])

In [18]:
word = randomWord()
inPlay = True

In [19]:
while inPlay:
    redraw_game_window()
    pygame.time.delay(10)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            inPlay = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                inPlay = False
        if event.type == pygame.MOUSEBUTTONDOWN:
            clickPos = pygame.mouse.get_pos()
            letter = buttonHit(clickPos[0], clickPos[1])
            if letter != None:
                guessed.append(chr(letter))
                buttons[letter - 65][4] = False
                if hang(chr(letter)):
                    if limbs != 5:
                        limbs += 1
                    else:
                        end()
                else:
                    print(spacedOut(word, guessed))
                    if spacedOut(word, guessed).count('_') == 0:
                        end(True)

_ _ _ _ _ _  _ S  _ _ _  _ _ _  _ _ _ _ _ 
_ _ _ _ _ _  I S  _ _ _  _ _ _  _ _ _ _ _ 
O _ _ _ _ _  I S  _ _ _  _ _ _  _ _ _ _ _ 
O R _ _ _ _  I S  _ _ _  _ _ _  _ _ _ _ _ 
O R A _ _ _  I S  _ _ _  _ _ _  _ _ A _ _ 
O R A _ _ E  I S  _ _ E  _ E _  _ _ A _ _ 
O R A N _ E  I S  _ _ E  N E _  _ _ A _ _ 
O R A N _ E  I S  _ _ E  N E _  _ L A _ _ 
O R A N _ E  I S  _ _ E  N E _  B L A _ _ 
O R A N _ E  I S  _ _ E  N E _  B L A C _ 
O R A N _ E  I S  _ _ E  N E _  B L A C K 
O R A N G E  I S  _ _ E  N E _  B L A C K 
O R A N G E  I S  T _ E  N E _  B L A C K 
O R A N G E  I S  T H E  N E _  B L A C K 
O R A N G E  I S  T H E  N E W  B L A C K 


error: video system not initialized

In [18]:
pygame.quit()